In [1]:
import sqlite3

db_file="fund_database.db"


In [3]:
#connexion à la base de données
try:
    # Connexion à la base de données SQLite
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Création de la table Clients
    cursor.executescript("""
                CREATE TABLE IF NOT EXISTS Clients (
                    id INTEGER PRIMARY KEY AUTOINCREMENT, 
                    name TEXT NOT NULL UNIQUE, 
                    age INTEGER NOT NULL, 
                    country TEXT NOT NULL,
                    email TEXT NOT NULL UNIQUE,
                    risk_profile TEXT NOT NULL,
                    registration_date TEXT NOT NULL,
                    investment_amount REAL NOT NULL,
                    manager_id INTEGER NOT NULL,
                    portfolio_id INTEGER NOT NULL
                );

                CREATE TABLE IF NOT EXISTS Managers (
                    id INTEGER PRIMARY KEY AUTOINCREMENT, 
                    name TEXT NOT NULL UNIQUE, 
                    age INTEGER NOT NULL, 
                    country TEXT NOT NULL,
                    email TEXT UNIQUE NOT NULL,
                    seniority TEXT NOT NULL,
                    investment_sector TEXT NOT NULL,
                );

                CREATE TABLE IF NOT EXISTS Manager_Strategies (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    manager_id INTEGER NOT NULL,
                    strategy TEXT NOT NULL,
                    FOREIGN KEY (manager_id) REFERENCES Managers(id),
                    UNIQUE (manager_id, strategy)  -- Evite les doublons
                );

                CREATE TABLE IF NOT EXISTS Manager_Clients (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    manager_id INTEGER NOT NULL,
                    client_id INTEGER NOT NULL,
                    FOREIGN KEY (manager_id) REFERENCES Managers(id),
                    FOREIGN KEY (client_id) REFERENCES Clients(id),
                    UNIQUE (manager_id, client_id)  -- Evite les doublons
                );

                CREATE TABLE IF NOT EXISTS Manager_Portfolios (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    manager_id INTEGER NOT NULL,
                    portfolio_id INTEGER NOT NULL,
                    FOREIGN KEY (manager_id) REFERENCES Managers(id) ON DELETE CASCADE,
                    FOREIGN KEY (portfolio_id) REFERENCES Portfolios(id) ON DELETE CASCADE,
                    UNIQUE (manager_id, portfolio_id)  -- Evite les doublons
                );

                CREATE TABLE IF NOT EXISTS Portfolios (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    manager_id INTEGER NOT NULL,
                    client_id INTEGER NOT NULL,
                    strategy TEXT NOT NULL,
                    investment_sector TEXT NOT NULL,
                    size INTEGER NOT NULL,
                    value REAL NOT NULL,
                    FOREIGN KEY (client_id) REFERENCES Clietns(id) ON DELETE CASCADE,
                    FOREIGN KEY (manager_id) REFERENCES Managers(id) ON DELETE CASCADE
                );


                CREATE TABLE IF NOT EXISTS Portfolio_Products (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    portfolio_id INTEGER NOT NULL,
                    product_id INTEGER NOT NULL,
                    
                    FOREIGN KEY (portfolio_id) REFERENCES Portfolios(id) ON DELETE CASCADE,
                    FOREIGN KEY (product_id) REFERENCES Products(id) ON DELETE CASCADE,
                    UNIQUE (portfolio_id, product_id)  -- Evite les doublons
                );
        """)
    print("✅ Toutes les tables ont été créées avec succès.")


    conn.commit()
    
except sqlite3.Error as e:

    print(f"Erreur SQLite : {e}")

finally:
    # Fermeture de la connexion
    if conn:
        conn.close()

✅ Toutes les tables ont été créées avec succès.


In [4]:
#Nouveau client
from data_collector import generate_precise_client, generate_random_client, manager_affiliation, create_manager, create_portfolio

from base_builder import Client, AssetManager, Portfolio



try: 
    db = sqlite3.connect(db_file)
    
    sortie=False

    # 🎯 Demande à l'utilisateur son choix
    choice = input("Voulez-vous créer un client aléatoire (1) ou manuel (2) ? 📌 Entrez 1 ou 2 : ")

    if choice == "1":
        client_data = generate_random_client()
    elif choice == "2":
        client_data = generate_precise_client()
    else:
        print("❌ Choix invalide ! Par défaut, on crée un client aléatoire.")
        client_data = generate_random_client()

    print(client_data)

    assigned_manager = manager_affiliation(client_data, db)

    if assigned_manager:
        client_data["manager_id"] = assigned_manager["id"]
        print(f"✅ Manager attribué : {assigned_manager['name']}")


    else:
        print("⚠️ Aucun asset manager opérant dans le pays du client, opérant la stratégie adéquate, et/ou ayant le niveau de séniorité adapté au client proposé n'a été trouvé.")
        choice = input("Voulez-vous que le fonds recrute un asset manager adéquat (Oui) ou préférez-vous annuler l'enregistrement du client (Non) 📌 Entrez 'Oui' ou 'Non' : ")
        
        if choice == "Oui":
            assigned_manager = create_manager(client_data, db)
            
        
        elif choice == "Non":
            client_data = None
            print("❌ Enregistrement du client annulé.")
            sortie=True
        
        else:
            print("❌ Choix invalide ! Par défaut, on trouve un Asset Manager adapté.")
            assigned_manager = create_manager(client_data, db)

        
        if not sortie:
            print(f"✅ Un Manager a le profil correspondant: {assigned_manager['name']} ")
            choice = input(" Devons-nous le recruter? 📌 Entrez 'Oui' ou 'Non' : ")

            if choice == "Oui":
                manager=AssetManager(**assigned_manager)
                client_data["manager_id"]=manager.save(db)
                print(f"✅ Manager {manager.name} recruté avec succès.")
            
            else:
                print("❌ Enregistrement du client annulé.")
                sortie=True
        

    ### Création du portefeuille à faire
    portfolio_data=create_portfolio(assigned_manager, client_data, db)  
    print("Portefeuille créé avec succès.")    

        
    if not sortie:

        ### Création du portefeuille à faire
        portfolio=Portfolio(**portfolio_data)
        client_data["portfolio_id"]=portfolio.save(db) # à modifier
        print("Portefeuille créé avec succès.")

        print(client_data["name"], "est à présent un client de 'Data Management Project'.")
        
        client = Client(**client_data)
        client.save(db)
        
    db.close


except sqlite3.Error as e:
    print(f"Erreur SQLite : {e}")


{'name': 'Gregory Russo', 'age': 29, 'country': 'Germany', 'email': 'gregory.russo@gmail.com', 'risk_profile': 'Low Risk', 'registration_date': '2022-06-22', 'investment_amount': 133214}
⚠️ Aucun asset manager opérant dans le pays du client, opérant la stratégie adéquate, et/ou ayant le niveau de séniorité adapté au client proposé n'a été trouvé.
✅ Un Manager a le profil correspondant: David Martinez 
✅ Manager David Martinez recruté avec succès.
Portefeuille créé avec succès.
{'name': 'Gregory Russo', 'age': 29, 'country': 'Germany', 'email': 'gregory.russo@gmail.com', 'risk_profile': 'Low Risk', 'registration_date': '2022-06-22', 'investment_amount': 133214, 'manager_id': 1, 'portfolio_id': 1}
✅ Client Gregory Russo ajouté avec succès.


In [32]:
from yahooquery import Screener

['ms_basic_materials',
    'ms_communication_services',
    'ms_consumer_cyclical',
    'ms_consumer_defensive',
    'ms_energy',
    'ms_financial_services',
    'ms_healthcare',
    'ms_industrials',
    'ms_real_estate',
    'ms_technology',
    'ms_utilities']

# Initialiser le screener
s = Screener()

# Récupérer les actions les plus échangées
ms_financial_services = s.get_screeners("ms_financial_services", 100)



# Afficher les résultats
for stock in ms_financial_services["ms_financial_services"]["quotes"]:
    print(stock["symbol"], "-", stock["shortName"], stock['market'])



BRK-A - Berkshire Hathaway Inc. us_market
BRK-B - Berkshire Hathaway Inc. New us_market
JPM - JP Morgan Chase & Co. us_market
V - Visa Inc. us_market
MA - Mastercard Incorporated us_market
BAC - Bank of America Corporation us_market
JPM-PD - J P Morgan Chase & Co Depositar us_market
JPM-PC - J P Morgan Chase & Co Depositar us_market
BML-PG - Bank of America Corporation Dep us_market
BML-PH - Bank of America Corporation Dep us_market
BAC-PK - Bank of America Corporation Dep us_market
BML-PL - Bank of America Corporation Dep us_market
BAC-PL - Bank of America Corporation Non us_market
BAC-PE - Bank of America Corporation Dep us_market
BML-PJ - Bank of America Corporation Dep us_market
BAC-PB - Bank of America Corporation Dep us_market
WFC - Wells Fargo & Company us_market
GS - Goldman Sachs Group, Inc. (The) us_market
HSBC - HSBC Holdings, plc. us_market
MS - Morgan Stanley us_market
WFC-PY - Wells Fargo & Company Depositar us_market
WFC-PL - Wells Fargo & Company 7.50% Non us_market
AXP